In [30]:
#CSV Bibliothek importieren 
import csv
#Pandas importieren und als pd bezeichnen
import pandas as pd
#Numpy importieren und als np bezeichnen 
import numpy as np
#Json Biliothek importieren
import json
#Obere Schranke für die Größe der Daten auf 100,000,000 setzen  
csv.field_size_limit(100000000)
#Maximale Anzahl von Zeilen und Spalten Zeigen 
pd.options.display.max_columns = None
pd.options.display.max_rows = None
#Extraktion der Domäne
import tldextract

***

### Pro_science domains

In [31]:
path = '../data/df_domains_pro_anti_science.tsv'

df_domains_pro_and_anti_science = pd.read_csv(path, sep='\t', error_bad_lines=False)

In [32]:
print('Shape: ', df_domains_pro_and_anti_science.shape)
df_domains_pro_and_anti_science.head()

Shape:  (1395, 4)


,domain,mediabias_proscience,mediabias_questionblesources,mediabias_conspiracy_psydoscience
0,Covid.gov,1.0,0.0,0.0
1,Drugs.com,1.0,0.0,0.0
2,News-Medical.net,1.0,0.0,0.0
3,SciTechDaily.com,1.0,0.0,0.0
4,aaas.org,1.0,0.0,0.0


In [33]:
df_proscience_domains = df_domains_pro_and_anti_science[df_domains_pro_and_anti_science['mediabias_proscience'] == 1]

print('Shape: ', df_proscience_domains.shape)
df_proscience_domains.head()

Shape:  (150, 4)


,domain,mediabias_proscience,mediabias_questionblesources,mediabias_conspiracy_psydoscience
0,Covid.gov,1.0,0.0,0.0
1,Drugs.com,1.0,0.0,0.0
2,News-Medical.net,1.0,0.0,0.0
3,SciTechDaily.com,1.0,0.0,0.0
4,aaas.org,1.0,0.0,0.0


In [34]:
list_proscience_domains = df_proscience_domains.domain.tolist()

In [35]:
def get_unique_items(l1):
    l1 = np.array(l1)
    l1 = np.unique(l1)
    l1 = l1.tolist()
    
    return l1

In [36]:
list_proscience_domains = get_unique_items(list_proscience_domains)
len(list_proscience_domains)

150

***

## Anti_science

***

In [37]:
df_domains_questionblesources = df_domains_pro_and_anti_science[df_domains_pro_and_anti_science['mediabias_questionblesources'] == 1]

print('Shape: ', df_domains_questionblesources.shape)
df_domains_questionblesources.head()

Shape:  (908, 4)


,domain,mediabias_proscience,mediabias_questionblesources,mediabias_conspiracy_psydoscience
150,100percentfedup.com,0.0,1.0,0.0
151,12minutos.com,0.0,1.0,0.0
152,2020conservative.com,0.0,1.0,0.0
153,24Jours.com,0.0,1.0,0.0
154,4Chan.org,0.0,1.0,0.0


In [38]:
list_domains_questionblesources = df_domains_questionblesources.domain.tolist()

In [39]:
df_domains_conspiracy_psydoscience = df_domains_pro_and_anti_science[df_domains_pro_and_anti_science['mediabias_conspiracy_psydoscience'] == 1]

print('Shape: ', df_domains_conspiracy_psydoscience.shape)
df_domains_conspiracy_psydoscience.head()

Shape:  (337, 4)


,domain,mediabias_proscience,mediabias_questionblesources,mediabias_conspiracy_psydoscience
1058,2020ElectionCenter.com,0.0,0.0,1.0
1059,21stcenturywire.com,0.0,0.0,1.0
1060,369news.net,0.0,0.0,1.0
1061,79Days.News,0.0,0.0,1.0
1062,911truth.org,0.0,0.0,1.0


In [40]:
list_domains_conspiracy_psydoscience = df_domains_conspiracy_psydoscience.domain.tolist()

In [41]:
list_antiscience_domains = []
list_antiscience_domains.extend(list_domains_conspiracy_psydoscience)
list_antiscience_domains.extend(list_domains_questionblesources)

list_antiscience_domains = get_unique_items(list_antiscience_domains)
len(list_antiscience_domains)

1245

## Polarisierungen berechnen
### TweetsCov19

---

In [13]:
path = '../data/df_tweetscov19_mai2020.tsv'

df_tweetscov19 = pd.read_csv(path, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Username','Timestamp' ,'Is_deleted', 'Domains'])

print('Shape :', df_tweetscov19.shape)
df_tweetscov19.head()

Shape : (1797419, 5)


,Tweet_Id,Username,Timestamp,Is_deleted,Domains
0,1256010447515516931,e03ffc23e75c403f5a2363a17a9d7ba0,2020-05-01 00:00:00+00:00,False,0
1,1256010444747108352,c3579f256ef8063db35cc6ec5a4e3653,2020-05-01 00:00:00+00:00,False,0
2,1256010446559039488,9897f9df6e48f764570e5e4535833703,2020-05-01 00:00:00+00:00,False,themarysue.com
3,1256010447507128324,68d58fda5bc46a45cde9aeb6cbd63585,2020-05-01 00:00:00+00:00,False,fox5dc.com
4,1256010447955701760,6524eab4c1031db6288c117374214f17,2020-05-01 00:00:00+00:00,False,0


In [42]:
def get_number_of_antiscience_domains(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        #d = tldextract.extract(domains_list[i])
        #d = d.domain          
        d = domains_list[i]
        
        if d in list_antiscience_domains:
            score -=  1
        
    return score

In [15]:
df_tweetscov19['Domains'] = df_tweetscov19['Domains'].astype(str)

df_tweetscov19['number_of_antisciens_domains'] = df_tweetscov19.apply(lambda row: get_number_of_antiscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [16]:
df_tweetscov19.head()

,Tweet_Id,Username,Timestamp,Is_deleted,Domains,number_of_antisciens_domains
0,1256010447515516931,e03ffc23e75c403f5a2363a17a9d7ba0,2020-05-01 00:00:00+00:00,False,0,0
1,1256010444747108352,c3579f256ef8063db35cc6ec5a4e3653,2020-05-01 00:00:00+00:00,False,0,0
2,1256010446559039488,9897f9df6e48f764570e5e4535833703,2020-05-01 00:00:00+00:00,False,themarysue.com,0
3,1256010447507128324,68d58fda5bc46a45cde9aeb6cbd63585,2020-05-01 00:00:00+00:00,False,fox5dc.com,0
4,1256010447955701760,6524eab4c1031db6288c117374214f17,2020-05-01 00:00:00+00:00,False,0,0


In [17]:
tweets_with_antiscience_domains = df_tweetscov19[df_tweetscov19['number_of_antisciens_domains'] < 0].shape

print('Number of tweets with at least one antiscience Domain: ', tweets_with_antiscience_domains[0])

Number of tweets with at least one antiscience Domain:  24691


In [43]:
def get_number_of_proscience_domains(domains):
    domains_list = domains.split()
    score = 0
    for i in range(0, len(domains_list)):
        
        #d = tldextract.extract(domains_list[i])
        #d = d.domain
       
            
        d = domains_list[i]
        if d in list_proscience_domains:
            score +=  1
        
    return score

In [19]:
df_tweetscov19['number_of_prosciens_domains'] = df_tweetscov19.apply(lambda row: get_number_of_proscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [20]:
df_tweetscov19.head()

,Tweet_Id,Username,Timestamp,Is_deleted,Domains,number_of_antisciens_domains,number_of_prosciens_domains
0,1256010447515516931,e03ffc23e75c403f5a2363a17a9d7ba0,2020-05-01 00:00:00+00:00,False,0,0,0
1,1256010444747108352,c3579f256ef8063db35cc6ec5a4e3653,2020-05-01 00:00:00+00:00,False,0,0,0
2,1256010446559039488,9897f9df6e48f764570e5e4535833703,2020-05-01 00:00:00+00:00,False,themarysue.com,0,0
3,1256010447507128324,68d58fda5bc46a45cde9aeb6cbd63585,2020-05-01 00:00:00+00:00,False,fox5dc.com,0,0
4,1256010447955701760,6524eab4c1031db6288c117374214f17,2020-05-01 00:00:00+00:00,False,0,0,0


In [21]:
tweets_with_proscience_domains = df_tweetscov19[df_tweetscov19['number_of_prosciens_domains'] > 0].shape

print('Number of tweets with at least one proscience domain: ', tweets_with_proscience_domains[0])

Number of tweets with at least one proscience domain:  7475


In [22]:

df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()

<ipython-input-22-5a729d8510a2>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetscov19_unique_user = df_tweetscov19.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()


In [23]:
number_of_unique_useres = df_tweetscov19_unique_user.shape

print('Number of unique users in TweetsCOV19: ', number_of_unique_useres[0])

Number of unique users in TweetsCOV19:  1064424


In [24]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['number_of_prosciens_domains']>0].head()

,number_of_antisciens_domains,number_of_prosciens_domains
Username,,
0002eb2aff20e181397afe8cdebdd395,0,1
0008667130fe3776676804ff59e8ce3b,0,1
0033ba10624ff9ac3a12b4c223cbc0ce,0,1
005696709556b8960c0598385492e8e0,0,1
0061dfef45d68a3fc2be1f8003bbe904,0,2


In [25]:
df_tweetscov19_unique_user['Sum_polarized_domains'] = df_tweetscov19_unique_user.apply(lambda row:  (row['number_of_prosciens_domains'] - row['number_of_antisciens_domains']), axis=1)

In [26]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['number_of_antisciens_domains']<0].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains
Username,,,
000238f324f5063164908c27a7f310cd,-1,0,1
00065a00b971b5e1629d504e0ddf2da0,-1,0,1
00084b153bbf7f4a8c1df3b638e07765,-2,0,2
000bcb01b05c0f275601b470ab115a06,-1,0,1
0013aaa4da0461c58d0b1306af75af02,-1,0,1


In [27]:
df_tweetscov19_unique_user['normalised_polarisation'] = df_tweetscov19_unique_user.apply(lambda row: (row['number_of_antisciens_domains'] + row['number_of_prosciens_domains']) / row['Sum_polarized_domains'] if row['Sum_polarized_domains'] != 0 else 0, axis=1)

In [28]:
df_tweetscov19_unique_user[df_tweetscov19_unique_user['number_of_antisciens_domains']<-2].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
Username,,,,
00b956ef2630a87295ee039edc94941d,-38,0,38,-1.0
015b920a4302d63b6a0e9cb2b0033ff5,-12,0,12,-1.0
017f9a97863fc2d5c1ed97f2c74c3ec8,-4,0,4,-1.0
01baae45b1a3710bf078f4e14c409ea0,-3,0,3,-1.0
01c85f37fda688a470fa0698665e31cb,-14,0,14,-1.0


In [29]:
users_with_with_atleast_one_polarized_domain = df_tweetscov19_unique_user[df_tweetscov19_unique_user['Sum_polarized_domains']>0]

print('Number of users with with at least one polarized domain: ',users_with_with_atleast_one_polarized_domain.shape[0])

Number of users with with at least one polarized domain:  13298


In [30]:
users_with_with_atleast_one_polarized_domain[users_with_with_atleast_one_polarized_domain['normalised_polarisation'] == 0].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
Username,,,,
006ca2433c6375610b279462e28fa322,-1,1,2,0.0
0573ad28d8f376c0f183c86c8747179c,-1,1,2,0.0
077e09bb652f5b61f270e9c5b6b82cd5,-1,1,2,0.0
07d06a39ed29f064c438655cf59d1274,-1,1,2,0.0
07d4b4dac03341b975acffd7e3d95746,-1,1,2,0.0


In [31]:
#df_tweetscov19_unique_user_without_zero_polarisation = df_tweetscov19_unique_user[df_tweetscov19_unique_user['normalised_polarisation'] != 0]

In [32]:
#number_of_science_polarized_users = df_tweetscov19_unique_user_without_zero_polarisation.shape

#print('Number of science polarized users in TWeetsCOV19: ', number_of_science_polarized_users[0])

In [33]:
number_of_proscience_polarized_users = df_tweetscov19_unique_user_without_zero_polarisation[df_tweetscov19_unique_user_without_zero_polarisation['normalised_polarisation'] > 0].shape

print('Number of proscience polarized users in TWeetsCOV19: ', number_of_proscience_polarized_users[0])

Number of proscience polarized users in TWeetsCOV19:  1693


In [34]:
number_of_antiscience_polarized_users = df_tweetscov19_unique_user_without_zero_polarisation[df_tweetscov19_unique_user_without_zero_polarisation['normalised_polarisation'] < 0].shape

print('Number of antiscience polarized users in TWeetsCOV19: ', number_of_antiscience_polarized_users[0])

Number of antiscience polarized users in TWeetsCOV19:  3286


****

In [35]:
df_tweetscov19_unique_user_without_zero_polarisation = df_tweetscov19_unique_user_without_zero_polarisation.reset_index()
df_tweetscov19_unique_user_without_zero_polarisation.head()

,Username,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
0,000df0c86153a1e9e65feca1051f8c1e,-1,0,1,-1.0
1,0013aaa4da0461c58d0b1306af75af02,-1,0,1,-1.0
2,0013ba9584257735e7b0380b5b831fe4,0,1,1,1.0
3,003ceef9d66817ec38c877b26b013d1d,0,1,1,1.0
4,005696709556b8960c0598385492e8e0,0,1,1,1.0


***

***

In [36]:
df_tweetscov19 = df_tweetscov19[['Tweet_Id', 'number_of_antisciens_domains', 'number_of_prosciens_domains']]


path = '../data/tweetscov19_tweets_science_polarized.tsv'

with open(path, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

In [33]:
users_with_with_atleast_one_polarized_domain = users_with_with_atleast_one_polarized_domain.reset_index()
users_with_with_atleast_one_polarized_domain.head()

pfad_tsv3 = '../data/df_tweetscov19_mai2020_science_polarized_users.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in users_with_with_atleast_one_polarized_domain.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

In [70]:
pfad_tsv3 = '../data/tweetscov19_science_polarized_users.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetscov19_unique_user_without_zero_polarisation.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

## Polarisierungen berechnen
### TweetsKB

***

In [44]:
pfad_tsv3 = '../data/df_tweetskb_januar2020.tsv'

df_tweetskb = pd.read_csv(pfad_tsv3, sep='\t', error_bad_lines=False, names=['Tweet_Id', 'Username', 'Timestamp' ,'Is_deleted', 'Domains'])

print('Shape :', df_tweetskb.shape)
df_tweetskb.head()

Shape : (1152147, 5)


,Tweet_Id,Username,Timestamp,Is_deleted,Domains
0,1212161513924022277,jeffvandermeer,2020-01-01 00:00:00+00:00,False,0
1,1212161515823853568,BarclaysFAWSL,2020-01-01 00:00:00+00:00,False,0
2,1212161513248743424,Reuters,2020-01-01 00:00:00+00:00,False,0
3,1212161513215021056,ArcSystemWorksU,2020-01-01 00:00:00+00:00,False,0
4,1212161514913849346,itdoesnteggs,2020-01-01 00:00:00+00:00,False,0


In [45]:
df_tweetskb['Domains'] = df_tweetskb['Domains'].astype(str)
df_tweetskb['number_of_antisciens_domains'] = df_tweetskb.apply(lambda row: get_number_of_antiscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [46]:
tweets_with_antiscience_domains = df_tweetskb[df_tweetskb['number_of_antisciens_domains'] < 0].shape

print('Number of tweets with at least one antiscience Domain: ', tweets_with_antiscience_domains[0])

Number of tweets with at least one antiscience Domain:  3553


In [47]:
df_tweetskb['number_of_prosciens_domains'] = df_tweetskb.apply(lambda row: get_number_of_proscience_domains(row['Domains']) if (row['Domains'] != '0') else 0, axis=1)

In [48]:
tweets_with_proscience_domains = df_tweetskb[df_tweetskb['number_of_prosciens_domains'] > 0].shape

print('Number of tweets with at least one antiscience Domain: ', tweets_with_proscience_domains[0])

Number of tweets with at least one antiscience Domain:  689


In [49]:

df_tweetskb_unique_user = df_tweetskb.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()

<ipython-input-49-532f45cc846d>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df_tweetskb_unique_user = df_tweetskb.groupby('Username')[ 'number_of_antisciens_domains', 'number_of_prosciens_domains'].sum()


In [50]:
unique_users = df_tweetskb_unique_user.shape

print('Number of unique users in TweetsKB: ', unique_users[0])

Number of unique users in TweetsKB:  886760


In [51]:
df_tweetskb_unique_user.head()

,number_of_antisciens_domains,number_of_prosciens_domains
Username,,
000000911a,0,0
000000_aaaaaaa,0,0
00000codename,0,0
00001smhq,0,0
00007soni,0,0


In [52]:
df_tweetskb_unique_user['Sum_polarized_domains'] = df_tweetskb_unique_user.apply(lambda row: (row['number_of_prosciens_domains'] - row['number_of_antisciens_domains']), axis=1)

In [53]:
df_tweetskb_unique_user[df_tweetskb_unique_user['number_of_antisciens_domains']<-2].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains
Username,,,
1776Stonewall,-7,0,7
623Sis,-3,0,3
AIIAmericanGirI,-7,0,7
ASimplePatriot,-7,0,7
AinsworthKeith,-3,0,3


In [54]:
df_tweetskb_unique_user['normalised_polarisation'] = df_tweetskb_unique_user.apply(lambda row: (row['number_of_antisciens_domains'] + row['number_of_prosciens_domains']) / row['Sum_polarized_domains'] if row['Sum_polarized_domains'] != 0 else 0, axis=1)

In [55]:
df_tweetskb_unique_users_with_atleast_one_polarized_domain= df_tweetskb_unique_user[df_tweetskb_unique_user['Sum_polarized_domains'] >0]

print('df tweetskb unique users with atleast one polarized domain: ', df_tweetskb_unique_users_with_atleast_one_polarized_domain.shape[0])

df tweetskb unique users with atleast one polarized domain:  2713


In [56]:
df_tweetskb_unique_users_with_atleast_one_polarized_domain[df_tweetskb_unique_users_with_atleast_one_polarized_domain['normalised_polarisation']==0].head()

,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
Username,,,,
EcoInternetDrGB,-1,1,2,0.0
FionasRevenge,-1,1,2,0.0
LotusOak2,-4,4,8,0.0
QAnonNotables,-1,1,2,0.0
downloadbucks,-1,1,2,0.0


In [57]:
#df_tweetskb_unique_user_without_zero_polarisation = df_tweetskb_unique_user[df_tweetskb_unique_user['normalised_polarisation'] != 0]

In [58]:
#number_of_science_polarized_users = df_tweetskb_unique_user_without_zero_polarisation.shape

#print('Number of science polarized users in TWeetsKB: ', number_of_science_polarized_users[0])

In [48]:
number_of_proscience_polarized_users = df_tweetskb_unique_user_without_zero_polarisation[df_tweetskb_unique_user_without_zero_polarisation['normalised_polarisation'] > 0].shape

print('Number of proscience polarized users in TWeetsKB: ', number_of_proscience_polarized_users[0])

Number of proscience polarized users in TWeetsKB:  2424


In [51]:
number_of_antiscience_polarized_users = df_tweetskb_unique_user_without_zero_polarisation[df_tweetskb_unique_user_without_zero_polarisation['normalised_polarisation'] < 0].shape

print('Number of antiscience polarized users in TWeetsKB: ', number_of_antiscience_polarized_users[0])

Number of antiscience polarized users in TWeetsKB:  10493


In [52]:
df_tweetskb_unique_user_without_zero_polarisation = df_tweetskb_unique_user_without_zero_polarisation.reset_index()
df_tweetskb_unique_user_without_zero_polarisation.head()

,Username,number_of_antisciens_domains,number_of_prosciens_domains,Sum_polarized_domains,normalised_polarisation
0,000000hSure,-1,0,1,-1.0
1,01splcheck,-1,0,1,-1.0
2,01sth02,0,1,1,1.0
3,0Bettymowery2,-1,0,1,-1.0
4,0Ludicris5,-2,0,2,-1.0


***

***

In [53]:
df_tweetskb = df_tweetskb[['Tweet_Id', 'number_of_antisciens_domains', 'number_of_prosciens_domains']]

path = '../data/tweetskb_tweets_science_polarized.tsv'

with open(path, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

In [59]:
df_tweetskb_unique_users_with_atleast_one_polarized_domain = df_tweetskb_unique_users_with_atleast_one_polarized_domain.reset_index()
df_tweetskb_unique_users_with_atleast_one_polarized_domain.head()

pfad_tsv3 = '../data/df_tweetskb_januar2020_science_polarized_users.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb_unique_users_with_atleast_one_polarized_domain.iterrows():
        tsv_writer.writerow(row.values.tolist())

***

In [98]:
pfad_tsv3 = '../data/tweetskb_science_polarized_users.tsv'

with open(pfad_tsv3, 'w', newline='', encoding="utf-8") as out_file:
    tsv_writer = csv.writer(out_file, delimiter="\t")
    for idx, row in df_tweetskb_unique_user_without_zero_polarisation.iterrows():
        tsv_writer.writerow(row.values.tolist())